In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import quad

def sum_of_n_gaussians_factory(n):
    """
    Returns a function f(x, *params) which is the sum of n Gaussians.
    params: (a1, x01, sigma1, a2, x02, sigma2, ..., an, x0n, sigman).
    """
    def multi_gaussian_n(x, *params):
        total = np.zeros_like(x, dtype=float)
        for i in range(n):
            A     = params[3*i + 0]
            x0    = params[3*i + 1]
            sigma = params[3*i + 2]
            total += A * np.exp(-((x - x0)**2)/(2*sigma**2))
        return total
    return multi_gaussian_n

def single_gaussian(x, A, x0, sigma):
    return A * np.exp(-((x - x0)**2)/(2*sigma**2))

def main():
    # -------------------------------------------------------------------------
    # 1. Read data (example: from .mca file). Adjust as needed for your format
    # -------------------------------------------------------------------------
    file_path = r"C:\Users\David1\Downloads\ciul.mca"

    with open(file_path, 'rb') as file:
        file_content = file.read().decode('latin1', errors='ignore')

    # The data in many .mca files appears after the '<<DATA>>' line:
    data_start_index = file_content.find('<<DATA>>') + len('<<DATA>>')
    data_string_full = file_content[data_start_index:]
    counts_full = [int(v) for v in data_string_full.split() if v.isdigit()]

    # Make sure we only keep up to 8192 channels (if the file is bigger)
    counts_trimmed = counts_full[:8192]

    # -------------------------------------------------------------------------
    # 2. Define a cut range if needed (channels to keep for fitting)
    # -------------------------------------------------------------------------
    cut_start = 1000
    cut_end   = 8000

    # x_values_cut will be from cut_start to cut_end-1
    x_values_cut = np.arange(cut_start, cut_end)
    counts_cut   = np.array(counts_trimmed[cut_start:cut_end])
    counts_cut = counts_cut*3600/91988.987
    # -------------------------------------------------------------------------
    # 3. Choose the number of Gaussians to fit
    # -------------------------------------------------------------------------
    n_peaks = 8

    # -------------------------------------------------------------------------
    # 4. Create the multi-Gaussian function for n_peaks
    # -------------------------------------------------------------------------
    multi_gaussian = sum_of_n_gaussians_factory(n_peaks)

    # -------------------------------------------------------------------------
    # 5. Provide initial guesses (3*n values = A, x0, sigma for each peak)
    #
    #    Adjust these guesses to approximate your actual peak positions & widths
    # -------------------------------------------------------------------------
    initial_params = [
        10,   1600,  10,    # Peak #1 guess
        10,   1750,  10,    # Peak #2 guess
        1100, 5900,  50,    # Peak #3 guess
        90,   6250,  30,    # Peak #4 guess
        10,   6350,  10,    # Peak #5 guess
        150,  6630,  30,    # Peak #6 guess
        30,   6770,  30,    # Peak #7 guess
        70,   4800,  20     # Peak #8 guess
    ]

    # -------------------------------------------------------------------------
    # 6. (Optional) Define parameter bounds:
    #    A >= 0, x0 in [cut_start, cut_end], sigma >= 0
    # -------------------------------------------------------------------------
    lower_bounds = []
    upper_bounds = []
    for i in range(n_peaks):
        lower_bounds += [0, cut_start, 0]
        upper_bounds += [np.inf, cut_end, np.inf]
    bounds = (lower_bounds, upper_bounds)

    # -------------------------------------------------------------------------
    # 7. Fit the data using curve_fit
    # -------------------------------------------------------------------------
    try:
        popt, pcov = curve_fit(
            multi_gaussian,
            x_values_cut,
            counts_cut,
            p0=initial_params,
            bounds=bounds,
            maxfev=10000
        )
        fit_successful = True
    except RuntimeError as e:
        print("Fit did not converge:", e)
        popt, pcov = None, None
        fit_successful = False

    # -------------------------------------------------------------------------
    # 8. If fit is successful, print parameters
    # -------------------------------------------------------------------------
    if fit_successful:
        perr = np.sqrt(np.diag(pcov))
        print("==== Fitted Parameters ====")
        for i in range(n_peaks):
            A_val     = popt[3*i + 0]
            x0_val    = popt[3*i + 1]
            sigma_val = popt[3*i + 2]
            A_err     = perr[3*i + 0]
            x0_err    = perr[3*i + 1]
            sigma_err = perr[3*i + 2]
            print(f"Peak #{i+1}:")
            print(f"  A     = {A_val:.2f} ± {A_err:.2f}")
            print(f"  x0    = {x0_val:.2f} ± {x0_err:.2f}")
            print(f"  sigma = {sigma_val:.2f} ± {sigma_err:.2f}")

        # ---------------------------------------------------------------------
        # 9. Compute area under each fitted Gaussian (numerical integration)
        # ---------------------------------------------------------------------
        print("\n==== Areas under each Gaussian ====")
        areas = []
        area_errors = []
        for i in range(n_peaks):
            Ai, x0i, si = popt[3*i], popt[3*i+1], popt[3*i+2]
            def gauss_i(xx):
                return single_gaussian(xx, Ai, x0i, si)
            # Integrate from cut_start to cut_end
            area_i, area_err_i = quad(gauss_i, cut_start, cut_end)
            areas.append(area_i)
            area_errors.append(area_err_i)
        
        for i, (area_i, err_i) in enumerate(zip(areas, area_errors), start=1):
            print(f"Peak #{i} area: {area_i:.2f} ± {err_i:.2e}")

        # (Optional) total area under the sum of n Gaussians
        def total_model(xx):
            return multi_gaussian(xx, *popt)
        total_area, total_err = quad(total_model, cut_start, cut_end)
        print(f"\nTotal area (sum of all {n_peaks} Gaussians): {total_area:.2f} ± {total_err:.2e}")

        # ---------------------------------------------------------------------
        # 10. Plot the data and the fit
        # ---------------------------------------------------------------------
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(x_values_cut, counts_cut, 'b.', label='Data (Cut)')

        # Plot the sum of all Gaussians
        fitted_y = multi_gaussian(x_values_cut, *popt)
        ax.plot(x_values_cut, fitted_y, 'r-', label=f'Sum of {n_peaks} Gaussians')

        # Optionally, plot each individual Gaussian
        for i in range(n_peaks):
            Ai, x0i, si = popt[3*i + 0], popt[3*i + 1], popt[3*i + 2]
            single_fit_y = single_gaussian(x_values_cut, Ai, x0i, si)
            ax.plot(x_values_cut, single_fit_y, '--')

        # Example reference lines at known or expected peak positions
        ax.axvline(x=1603, color='blue',    linestyle='--', linewidth=2, label='Fe55 Ka')
        ax.axvline(x=1758, color='red',     linestyle='--', linewidth=2, label='Fe55 Kb')
        ax.axvline(x=5885, color='orange',    linestyle='--', linewidth=2, label='Cd109 Ka')
        ax.axvline(x=6250, color='purple',  linestyle='--', linewidth=2, label='Cd109 Kb1')
        ax.axvline(x=6350, color='purple',  linestyle='--', linewidth=2, label='Cd109 Kb2')
        ax.axvline(x=6633, color='green',  linestyle='--', linewidth=2, label='Cd109 Kb3')
        ax.axvline(x=6769, color='green',   linestyle='--', linewidth=2, label='Cd109 Kb4')

        ax.set_xlabel('Channel Number')
        ax.set_ylabel('Counts\Hour')
        #ax.set_title(f'Fitting {n_peaks} Gaussians to {file_path}')
        ax.legend()
        ax.grid(True)
        plt.show()

    else:
        print("Fitting failed. No results to show.")

if __name__ == "__main__":
    main()
